# CLASSIFICATORE SVM LINEARE CON INPUT DI WORD EMBEDDINGS

Classificatore basato su svm lineare che prende in input i word embeddings di parole italiane, più specificamente dei word embedding sviluppati per EvalIta 2018 dall'Italian NLP Lab. 

***

In [42]:
import numpy as np 
import re 
import os

### Caricamento Word Embeddings

Il file txt dei word embedding è stato ottenuto attraverso l'elaborazione di un file sqlite. Per ogni riga (la quale rappresenta una parola) all'interno del file, si crea un record in un dizionario con chiave parola e come valore il vattore che rappresenta il word embedding.

In [26]:
#funzione per il caricamento degli embedding itwac
def load_embeddings():
    embeddings_dict = dict()
    for line in open('../data/embeddings/itwac32.txt', encoding='utf-8'):
        splitted = line.strip().split('\t')
        word = splitted[0]
        embedding = splitted[1:]
        embedding = [float(val) for val in embedding]
        embeddings_dict[word] = np.asarray(embedding)
    return embeddings_dict

embeddings = load_embeddings()


In [38]:
embeddings['amico']

array([-0.02069134,  0.09119736,  0.25785723, -0.23561105, -0.28197852,
       -0.13193955, -0.13197723, -0.05229089, -0.28881341,  0.06564969,
       -0.30802506,  0.11779311, -0.03571652, -0.08748714, -0.24729131,
        0.2577146 ,  0.11925782, -0.27795964,  0.20498367,  0.08414506,
        0.08175091, -0.05181665, -0.34403449, -0.05261306,  0.08858071,
       -0.09748928,  0.12879393,  0.04387102, -0.04690454,  0.08181785,
        0.321078  ,  0.01658573])

### Normalizzazione del testo 

Le parole all'interno dei post devono essere normalizzate in questo modo:

Numeri:
- interi tra 0 e 2100 possono essere mantenuti così come sono
- i numeri interi maggiori di 2100 diventano una stringa specifica con un numero che rappresenta il numero di cifre
- se non si tratta di numeri interi, si convertono le cifre all'interno della stringa in questa sequenze @Dg

Parole:
- le parole che iniziano con una lettera maiuscola devono avere prima parola maiuscola e le altre minuscole
- le parole che iniziano con una lettera minuscola devono avere tutti i caratteri minuscolo

In [41]:
def digit_norm(word):
    try:
        val = int(word)
    except:
        normalized = re.sub('\d', '@Dg', word)
        return normalized
    if val >= 0 and val<1200:
        return str(val)
    else:
        return f'DIGLEN_{str(len(str(val)))}'


def normalize(word):
    if "http" in word or ("." in word and "/" in word):
        return str("___URL___")
    if len(word)>26:
        return "__LONG-LONG__"
    digits = digit_norm(word)
    if digits != word:
        word = digits
    if word[0].isupper():
        word = word.capitalize()
    else:
        word = word.lower()
    return word



In [57]:
def get_tokens(doc_path):
    doc_tokens = []
    skip_lines = 0
    first = False
    for line in open(doc_path, "r", encoding="utf-8"):
        splitted = line.strip().split("\t")
        if line[0].isdigit():
            if "-" in splitted[0]:
                word=splitted[1]
                skip_lines = len(splitted[0].split('-'))
                first = True
            if skip_lines == 0:
                print(line[0])
                word = splitted[1]
                lemma=splitted[2]
                pos=splitted[3]
                token = {'word': word, 'lemma': lemma, 'pos': pos}
                doc_tokens.append(token)
            if skip_lines != 0 and first == True:
                lemma = splitted[2]
                pos = splitted[3]
                token = {'word': word, 'lemma': lemma, 'pos': pos}
                doc_tokens.append(token)
                skip_lines-=1
                first = False
            else: 
                skip_lines-=1
    return doc_tokens

def create_set(type):
    annotated_posts = []
    for doc in os.listdir("../data/UD_annotation"):
        if type in doc: 
            doc_path = "../data/UD_annotation/" + doc    
            doc_tokens = get_tokens(doc_path)
            annotated_posts.append(doc_tokens)
    return annotated_posts


print(get_tokens("../data/UD_annotation/training#125642756147265536#0#0#TW-SENTIPOLC.conllu"))

1
1
[{'word': 'Le', 'lemma': 'il', 'pos': 'DET'}, {'word': 'rendersi', 'lemma': '_', 'pos': '_'}, {'word': 'si', 'lemma': 'si', 'pos': 'PRON'}]
